# Generate epidemic dataset

split dataset to two parts:

- 2020-01-29 to 2021-08-30 as training set, coresponding to twitter_dataset

- after 2021-09-01 as predicting set, use to predict

import modules and start spark session

In [29]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

appName = "Generate Twitter Dataset"
spark = SparkSession.builder.appName(appName).getOrCreate()

read dataset

In [30]:
data = spark.read.csv("../datasets/epidemic_raw.csv", header=True, inferSchema=True)

select needed columns, fill null with 0

In [31]:
data = data.select(
    col("date").cast("date").alias("date"),
    col("cases").cast("int").alias("cases"),
    col("newCases").cast("int").alias("newCases"),
    col("deaths").cast("int").alias("deaths"),
    col("positiveTests").cast("int").alias("positive"),
    col("negativeTests").cast("int").alias("negative"),
).fillna(0)

split dataset with date

In [32]:
dataTrain = data.filter(col("date") >= "2020-01-29").filter(col("date") <= "2020-08-30")
dataPred  = data.filter(col("date") >= "2020-08-31")
dataTrain.show(2)
dataPred.show(2)

+----------+-----+--------+------+--------+--------+
|      date|cases|newCases|deaths|positive|negative|
+----------+-----+--------+------+--------+--------+
|2020-01-29|    5|       0|     0|       0|       0|
|2020-01-30|    6|       1|     0|       0|       0|
+----------+-----+--------+------+--------+--------+
only showing top 2 rows

+----------+-------+--------+------+--------+--------+
|      date|  cases|newCases|deaths|positive|negative|
+----------+-------+--------+------+--------+--------+
|2020-08-31|6045457|   36487|183472| 7823449|82147658|
|2020-09-01|6089506|   44049|184563| 7871728|82994861|
+----------+-------+--------+------+--------+--------+
only showing top 2 rows



In [34]:
dataTrain.toPandas().to_csv("../datasets/epidemic_train.csv", header=True)
dataPred.toPandas().to_csv("../datasets/epidemic_pred.csv", header=True)